# Capstone Project - Comparing Neighborhoods (New York vs Toronto)

## 1. Introduction/Business problem

Travel is a multibillion-dollar industry and millions of people travel for work or pleasure. But as all seasoned travelers know it is good to plan your trip well in advance. There are basically two types of travelers. First, those that want to experience the comforts of what they are used to at home, at their new destination. For example, if you are from New York and love Starbucks coffee and McDonalds hamburgers, you may want to visit similar venues in the city that you are traveling to. Second, you have the adventurous type that wants to experience new things, such as types of cuisines you do not find at home. Again, if you are from New York and you go a foreign city, you may want to try local cuisines and visit venues that you do not find at home.

There are many apps and web sites that will recommend restaurants and venues with ratings of how well they were perceived. There are not that many that compare neighborhoods on how similar of dissimilar they are to your home city.  It could be enlightening to someone to be able to see maps of the origination city and the destination city with similar neighborhoods clustered together based on an analysis of the top venues in each neighborhood. For example, if you love coffee shops and bakeries, you may want to visit similar neighborhoods in the destination city that have an abundance of good coffee shops and bakeries as well. The other side of the coin is that somebody else may want to avoid such neighborhoods and visit neighborhoods with a different character. 

The result of a joint clustering of neighborhoods in the two cities will be presented as a map of the origination city and the destination city with color marked neighborhoods. It will then be easy for the traveler to identify similar neighborhoods (having the same color) and also dissimilar neighborhoods (having different colored markers). A second analysis will exploit hierarchical clustering and present the result as a dendogram so that the traveler can gain insight into how similar of dissimilar two neighborhoods are. This analysis will present the traveler with interesting and useful information that will make the planned trip more enjoyable.

In this project the analysis described above will compare Downtown New York to downtown Toronto. 


## 2. Data

There are various data sources that will be exploited in the analysis. First, we need neighborhood data for New York and Toronto:

-	https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
-	https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

For the Toronto data set we also need to add latitude and longitude

-	https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

that need to be joined with the postal code of Canada data set. These data sets need to be read and transformed into pandas data frames. The resulting data sets will contain 4 features: borough, neighborhood, latitude, and longitude.

The next step is to exploit the Foursquare api 

-   https://api.foursquare.com/v2/venues/explore?

to explore the neighborhoods and to create a data frame that will contain the neighborhood name the top 10 venues for each neighborhood (10 features containing the 1st to 10th top venues). Examples of venues are Coffee Shop, Grocery Store, Japanese restaurant, etc. The result will be two data sets: new_york_venues and toronto_venues that will be combined into one data set ny_toronto_venues.

This is the data set that will then be used in the cluster analysis that will follow.

As a last step the latitude and longitude information will be added to be able to create the map. 
